# Overview

- this is a baseline model to determine immediate performance
- will use gradient boost modelling using package lightgbm
- no data cleaning feature engineering to be used

In [15]:
# General Imports

import numpy as np
import pandas as pd
import pickle

import os, gc, sys, warnings, random, psycopg2

In [16]:
# Functions

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

In [17]:
SEED = 42
LOCAL_TEST = False
seed_everything(SEED)
TARGET = 'meter_reading'

In [29]:
# open pickle files

train_df = pd.read_pickle('train_df.pkl')
test_df = pd.read_pickle('test_df.pkl')
submission = pd.read_pickle('submission.pkl')

In [19]:
#data check#data check
print("Main:",list(train_df),train_df.info())
print("#" * 20)
print("Main:",list(test_df),test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17684654 entries, 0 to 17684653
Data columns (total 23 columns):
 #   Column              Dtype         
---  ------              -----         
 0   building_id         int16         
 1   site_id             int8          
 2   meter               int8          
 3   timestamp           datetime64[ns]
 4   meter_reading       float64       
 5   primary_use         int8          
 6   square_feet         int32         
 7   year_built          int8          
 8   floor_count         int8          
 9   air_temperature     float64       
 10  cloud_coverage      float16       
 11  dew_temperature     float64       
 12  precip_depth_1_hr   float16       
 13  sea_level_pressure  float64       
 14  wind_direction      float16       
 15  wind_speed          float64       
 16  DT_month            int8          
 17  DT_week_y           int8          
 18  DT_day_y            int8          
 19  DT_hour             int8          
 20  

In [20]:
# Check memory usage

In [21]:
# Model Parameters
import lightgbm as lgb

lgb_params = {
    
    'objective':'regression',
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'n_jobs':-1,
    'learning_rate':0.05,
    'num_leaves': 2**8,
    'max_depth':-1,
    'tree_learner':'serial',
    'colsample_bytree': 0.7,
    'subsample_freq':1,
    'subsample':0.7,
    'n_estimators':1000,
    'max_bin':255,
    'verbose':-1,
    'seed': SEED,
    'early_stopping_rounds':200,
    
}

In [22]:
#prepare training data

#choose columns to remove
remove_columns = [TARGET,'timestamp']
#get feature columns using list comprehension to remove columns - list(train_df)
feature_columns = [col for col in list(train_df) if col not in remove_columns]

#use np.log1 for distribution
#label is target

tr_data = lgb.Dataset(train_df[feature_columns],label=np.log1p(train_df[TARGET]))
eval_sets = [tr_data]

In [23]:
#train model

seed_everything(15)
lgb_params['seed'] = 15

estimator = lgb.train(
            lgb_params,
            tr_data,
            valid_sets = eval_sets,
            verbose_eval = 100,
      
)

/home/douglas/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/douglas/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds
[100]	training's rmse: 1.00401
[200]	training's rmse: 0.862542
[300]	training's rmse: 0.805057
[400]	training's rmse: 0.761733
[500]	training's rmse: 0.732619
[600]	training's rmse: 0.70861
[700]	training's rmse: 0.689736
[800]	training's rmse: 0.673566
[900]	training's rmse: 0.660451
[1000]	training's rmse: 0.648356
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 0.648356


In [34]:
# Model Predictor

test_df = test_df[feature_columns]

predictions = []

predictions = list(np.expm1(estimator.predict(test_df[feature_columns])))

#Add to submission file

submission = pd.read_pickle('submission.pkl')

In [35]:
predictions

[10.487716140740233,
 4.339515511962171,
 5.258203689296575,
 1.4881454203557565,
 5.838386318688497,
 9.24966901785005,
 1.7098284384054967,
 3.1462907467176517,
 6.497794714683874,
 213.53408650400615,
 7.895624642737227,
 2.8750474291926023,
 4.8677613414603425,
 11.908949113279126,
 7.753131357023403,
 5.114610351034153,
 3.0041479695950497,
 18.338798940773692,
 5.619394169969854,
 153.5980752532806,
 3.863030262958884,
 72.67092748913421,
 9.250059795345502,
 5.007368859564545,
 6.634025100617918,
 4.764895630473426,
 5.214623607852516,
 3.6703261476489812,
 2.0648938425023005,
 8.552132537708848,
 5.902781726323275,
 5.95484797586333,
 3.069563814553207,
 5.226865838016012,
 3.7824226935332304,
 83.1417767022585,
 4.621735335194024,
 17.75632722289158,
 4.235165973361069,
 47.0300592453614,
 7.276995396305655,
 134.86943975076312,
 4.184743790733101,
 5.772688584737814,
 4.6518509484852775,
 4.077454880244208,
 3.131170271279001,
 3.22943052117855,
 8.53917574820508,
 4.64917720

In [36]:
submission['meter_reading'] = predictions

ValueError: Length of values (36491273) does not match length of index (41697600)